In [1]:
import tensorflow as tf

print(tf.__version__)
a = tf.constant([2, 3, 5], dtype=tf.float32)

b1 = a / tf.reduce_sum(a)
print(b1)

b2 = tf.nn.softmax(a)
print(b2)

2022-06-26 22:07:13.201292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-26 22:07:13.201316: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.1
tf.Tensor([0.2 0.3 0.5], shape=(3,), dtype=float32)
tf.Tensor([0.04201007 0.11419519 0.8437947 ], shape=(3,), dtype=float32)


2022-06-26 22:07:17.293311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-26 22:07:17.294001: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-26 22:07:17.294432: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-26 22:07:17.294730: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-26 22:07:17.294989: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

1. 经过指数形式的Softmax函数能够将差距大的数值距离拉的更大
2. 在深度学习中通常使用反向传播求解梯度进而使用梯度下降进行参数更新的过程，而指数函数在求导的时候比较方便

In [2]:
import math

print(math.exp(2)/(math.exp(2) +  math.exp(3) + math.exp(5)))

0.042010066134066056


指数函数的曲线写了逐渐增大虽然能够将输出值拉开距离，但是也带来了缺点，当$z_i$值非常大的话，计算得到的数值也会变得非常大，数值可能溢出

In [3]:
import numpy as np

scores = np.array([123, 456, 789])
softmax = np.exp(scores) / np.sum(np.exp(scores))
print(softmax)

[ 0.  0. nan]


/tmp/ipykernel_20707/4703097.py:4: RuntimeWarning: overflow encountered in exp
  softmax = np.exp(scores) / np.sum(np.exp(scores))
/tmp/ipykernel_20707/4703097.py:4: RuntimeWarning: invalid value encountered in true_divide
  softmax = np.exp(scores) / np.sum(np.exp(scores))


针对数值溢出有其对应的优化方法，将每一个输出值减去输出值中最大值

$D = max(z)$

$softmax(z_i)=\frac{e^{z_i-D}}{{\sum_{c=1}^C}e^{z_c-D}}$

In [4]:
import numpy as np

scores = np.array([123, 456, 789])
scores -= np.max(scores)
p = np.exp(scores) / np.sum(np.exp(scores))
p

array([5.75274406e-290, 2.39848787e-145, 1.00000000e+000])

这里需要注意一下，当使用Softmax函数作为输出节点的激活函数的时候，一般使用交叉熵作为损失函数。由于Softmax函数的数值计算过程中，很容易因为输出节点的输出值比较大而发生数值溢出的现象，在计算交叉熵的时候也可能会出现数值溢出的问题。为了数值计算的稳定性，TensorFlow提供了一个统一的接口，将Softmax与交叉熵损失函数同时实现，同时也处理了数值不稳定的异常，使用TensorFlow深度学习框架的时候，一般推荐使用这个统一的接口，避免分开使用Softmax函数与交叉熵损失函数。
```python
import tensorflow as tf

print(tf.__version__) # 2.0.0
tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits = False)
```
其中y_true代表了One-hot编码后的真实标签，y_pred表示网络的实际预测值：
1. 当from_logits设置为True时，y_pred表示未经Softmax函数的输出值；
2. 当from_logits设置为False时，y_pred表示为经过Softmax函数后的输出值；

In [6]:
import tensorflow as tf

print(tf.__version__)
z = tf.random.normal([2, 10]) # 构造2个样本的10类别输出的输出值
y = tf.constant([1, 3]) # 两个样本的真实样本标签是1和3
y_true = tf.one_hot(y, depth = 10) # 构造onehot编码

# 输出层未经过Softmax激活函数,因此讲from_logits设置为True
loss1 = tf.keras.losses.categorical_crossentropy(y_true, z, from_logits = True)
loss1 = tf.reduce_mean(loss1)
print(loss1) # tf.Tensor(2.6680193, shape=(), dtype=float32)

y_pred = tf.nn.softmax(z)
# 输出层经过Softmax激活函数,因此讲from_logits设置为True
loss2 = tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits = False)
loss2 = tf.reduce_mean(loss2)
print(loss2) # tf.Tensor(2.668019, shape=(), dtype=float32)

2.9.1
tf.Tensor(3.2160025, shape=(), dtype=float32)
tf.Tensor(3.2160022, shape=(), dtype=float32)


2022-06-26 22:24:53.410487: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-26 22:24:53.412454: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-26 22:24:53.412970: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-26 22:24:53.413359: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-26 22:24:53.413652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [7]:
# 计算交叉熵损失值
import tensorflow as tf

print(tf.__version__) # 2.0.0
z = tf.constant([4, -4, 3], dtype = tf.float32)
y_hat = tf.nn.softmax(z)
y = tf.one_hot(2, depth = 3)
print("x:",z)
print("y_hat:", y_hat)
print("y:", y)

CE = tf.keras.losses.categorical_crossentropy(y, z, from_logits = True)
CE = tf.reduce_mean(CE)

print("cross_entropy:", CE)

2.9.1
x: tf.Tensor([ 4. -4.  3.], shape=(3,), dtype=float32)
y_hat: tf.Tensor([7.3087937e-01 2.4518272e-04 2.6887551e-01], shape=(3,), dtype=float32)
y: tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)
cross_entropy: tf.Tensor(1.3135068, shape=(), dtype=float32)


In [8]:
# 计算梯度值
import tensorflow as tf

print(tf.__version__) # 2.0.0
z = tf.constant([4, -4, 3], dtype = tf.float32)

# 构造梯度记录器
with tf.GradientTape(persistent = True) as tape:
    tape.watch([z])
    # 前向传播过程
    y = tf.one_hot(2, depth=3)
    CE = tf.keras.losses.categorical_crossentropy(y, z, from_logits=True)
    CE = tf.reduce_mean(CE)

dCE_dz = tape.gradient(CE, [z])[0]
print(dCE_dz)

2.9.1
tf.Tensor([ 7.3087937e-01  2.4518272e-04 -7.3112452e-01], shape=(3,), dtype=float32)
